# AFSK Demodulator
## Step 1: Basic AXI Streaming

This is a Pynq portion of the AFSK demodulator project.  We will be using the FPGA overlay that we created in Vivado.

At this point we have created the bitstream for "project_01" and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

We need to load two Python pynq modules, one for managing the overlay, and the others for managing DMA buffers. We also need to load `numpy` as Pynq relies on the numpy data types.  We then load the bitstream and get a handle to the dma controller.

In [15]:
from pynq import Overlay, Xlnk
import numpy as np

overlay = Overlay('project_01.bit')
dma = overlay.demodulator.afsk_dma

Now we need to construct the arrays used to transfer the data via DMA and initialize the data.

In [18]:
xlnk = Xlnk()
out_buffer = xlnk.cma_array(shape=(264,), dtype=np.int16)
in_buffer = xlnk.cma_array(shape=(264,), dtype=np.int16)

for i in range(264):
    out_buffer[i] = 16

We then send the data, initiate a read, wait for both transfers to complete, and then print the output.

In [19]:
dma.sendchannel.transfer(out_buffer)
dma.recvchannel.transfer(in_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()

print(in_buffer, len(in_buffer))

[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10] 264


Note that our FPGA project did exactly what it was designed to do.

$y[d] = \frac{5}{8} x[d]$

## Next Steps

Now that we have an overlay for Pynq built and working, we will take the AFSK demodulator and start moving components of that into the FPGA fabric.

As a quick refresher, here are the processing steps in the demodulator:

 1. Band-pass filter the audio data to remove as much noise as possible.
 1. Use a comparator to convert the analog signal into a bit stream.
 1. Delay the binary data.
 1. Multiply the bit stream with the delayed version using XOR (comb filter).
 1. Low-pass filter the PWM-like output of the comb filter.
 1. Use a comparator to convert the comparator output into a bit stream.
 1. Use a digital PLL to perform clock recovery.
    1. Implement an IIR filter for the PLL lock and loop filters.
    1. Implement a hysteresis function for the PLL lock control.
 1. Decode the NRZI bitstream into data.
 1. Decode the data bitstream into HDLC packets.
    1. Implement a CRC-CCITT function to compute the packet checksum.

## Current Implementation

Below is the current implementation of the AFSK demodulator in Python.

In [16]:
import sys
sys.path.append('../../base')

import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read
from DigitalPLL import DigitalPLL
from HDLC import HDLC
from AX25 import AX25
import time

class fir_filter(object):
    def __init__(self, coeffs):
        self.coeffs = coeffs
        self.zl = lfiltic(self.coeffs, 32768.0, [], [])
    def __call__(self, data):
        start_time = time.time()
        result, self.zl = lfilter(self.coeffs, 32768.0, data, -1, self.zl)
        stop_time = time.time()
        sw_exec_time = stop_time - start_time
        print('Software FIR execution time: ',sw_exec_time)
        return result

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result


audio_file = read('../../base/TNC_Test_Ver-1.102-26400-1sec.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]
delay = 12 # ~446us

bpf_coeffs = np.array(firwin(141, [1100.0/(sample_rate/2), 2300.0/(sample_rate/2)], width = None,
        pass_zero = False, scale = True, window='hann') * 32768, dtype=int)

bpf = fir_filter(bpf_coeffs)

lpf_coeffs = np.array(firwin(101, [760.0/(sample_rate/2)], width = None,
        pass_zero = True, scale = True, window='hann') * 32768, dtype=int)

lpf = fir_filter(lpf_coeffs)

filter_delay = len(bpf_coeffs)//2

# Band-pass filter the audio data
f = np.append(bpf(audio_data[:26400]), bpf(np.zeros(filter_delay)))[filter_delay:]
# Digitize the data
d = np.array([int(x > 0) for x in f])
# Delay the data
a = d[delay:]
# XOR the digitized data with the delayed version
x = np.logical_xor(d[:0-delay], a)
# Low-pass filter the PWM signal
c = np.append(lpf(x-0.5), lpf(np.zeros(len(lpf_coeffs)//2)))[len(lpf_coeffs)//2:]
# Digitize the tone transistions
dx = np.array([int(x > 0) for x in c])
# Create the PLL
pll = DigitalPLL(sample_rate, 1200.0)

locked = np.zeros(len(dx), dtype=int)
sample = np.zeros(len(dx), dtype=int)

# Clock recovery
for i in range(len(dx)):
    sample[i] = pll(dx[i])
    locked[i] = pll.locked()
    
nrzi = NRZI()

data = [int(nrzi(x)) for x,y in zip(dx, sample) if y]

hdlc = HDLC()

for b,s,l in zip(dx, sample, locked):
    if s:
        packet = hdlc(nrzi(b), l)
        if packet is not None:
            print(AX25(packet[1]))


Software FIR execution time:  0.029716014862060547
Software FIR execution time:  0.0024309158325195312
Software FIR execution time:  0.022467613220214844
Software FIR execution time:  0.002061605453491211
KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!?
